In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# parameter keys
PARAM_YEAR = 'ww_x_PERIODE_ACAD'
PARAM_STATUS = 'ww_x_PERIODE_PEDAGO'
PARAM_SEMESTER = 'ww_x_HIVERETE'
PARAM_GPS = 'ww_x_GPS'

In [3]:
FILTER_BASE_URL = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847'
GPS_BASE_URL = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_b_list=1&ww_x_UNITE_ACAD=249847'

In [4]:
STATUS_B1 = "Bachelor semestre 1"
STATUS_B6 = "Bachelor semestre 6"

In [5]:
# create dict from dropdown menus
def create_dict(field_list):
    field_dict = {}
    for i in range(len(field_list)):
        if field_list[i].string != None:
            field_dict[field_list[i].string] = field_list[i]["value"]
    return field_dict

r = requests.get(FILTER_BASE_URL)
soup = BeautifulSoup(r.content, 'html.parser')

year_dict = create_dict(soup.body.find(id="filtre").find_all('tr')[1].find_all('option'))
status_dict = create_dict(soup.body.find(id="filtre").find_all('tr')[2].find_all('option'))
sem_dict = create_dict(soup.body.find(id="filtre").find_all('tr')[3].find_all('option'))

In [7]:
def is_valid_gps(gps):
    return gps != "-1"

def extract_gps(content):
    soup = BeautifulSoup(content, 'html.parser')
    elements = soup.find_all('a', class_='ww_x_GPS')
    for element in elements:
        raw_info = element.attrs['onclick']
        gps = raw_info.split("'")[1].split('=')[1]
        if is_valid_gps(gps):
            return gps

In [8]:
# TODO: query records by gps & store data to the global dict
def handle_gps(gps, year, status, semester):
    url = GPS_BASE_URL + param_in_url(PARAM_GPS, gps)
    # r = requests.get(url)
    # print(r.content)  

In [12]:
# bachelors
for year, year_val in year_dict.items():
    for status, status_val in status_dict.items():
        if status == STATUS_B1 or status == STATUS_B6:
            for semester, sem_val in sem_dict.items():
                # construct urls for query
                payload_filter = {PARAM_YEAR: year_val, 
                                  PARAM_STATUS: status_val, 
                                  PARAM_SEMESTER: sem_val}
                r = requests.get(FILTER_BASE_URL, params=payload_filter)
                gps = extract_gps(r.content)
                if gps != None:
                    print(year, status, semester, gps, sep=", ")            
                    # query by gps & extract data to the global dict
                    handle_gps(gps, year, status, semester)

2015-2016, Bachelor semestre 6, Semestre de printemps, 1650772010
2015-2016, Bachelor semestre 1, Semestre d'automne, 1897032870
2011-2012, Bachelor semestre 6, Semestre de printemps, 213618020
2011-2012, Bachelor semestre 1, Semestre d'automne, 736308968
2010-2011, Bachelor semestre 6, Semestre de printemps, 109577031
2010-2011, Bachelor semestre 1, Semestre d'automne, 357704486
2012-2013, Bachelor semestre 6, Semestre de printemps, 357704613
2012-2013, Bachelor semestre 1, Semestre d'automne, 1378362092
2014-2015, Bachelor semestre 6, Semestre de printemps, 1378362238
2014-2015, Bachelor semestre 1, Semestre d'automne, 1744377893
2007-2008, Bachelor semestre 6, Semestre de printemps, 19561998
2007-2008, Bachelor semestre 1, Semestre d'automne, 71297531
2016-2017, Bachelor semestre 6, Semestre de printemps, 1744378039
2016-2017, Bachelor semestre 1, Semestre d'automne, 2021043255
2013-2014, Bachelor semestre 6, Semestre de printemps, 736309090
2013-2014, Bachelor semestre 1, Semestre 